In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences, sequence
from keras.models import Sequential
from keras.layers import Dense, Flatten, CuDNNLSTM, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras import optimizers,metrics,layers
import matplotlib.pyplot as plt
## Plot
# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Other
import re
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

Using TensorFlow backend.


In [2]:
w2vdf = pd.read_csv('../input/word2vec.csv')

In [3]:
w2vdf.head()

,Unnamed: 0,0,1
0,0,the,[2.62178 2.4556017 2.081975 2.006189 2.915...
1,1,to,[2.392987 2.33495 1.8192794 1.637305 2.500...
2,2,i,[2.2249963 2.1055882 1.9827801 1.6818206 2.346...
3,3,and,[2.3818252 2.2474759 1.9917579 1.923944 2.770...
4,4,a,[2.4556684 2.0998123 1.8610787 1.8022392 2.502...


In [4]:
vocab = ['unk']
g = dict()
g['unk'] = np.array([0]*300)
for i in range(w2vdf.shape[0]):
    g[w2vdf.iloc[i,1]] = np.fromstring(w2vdf.iloc[i,2][1:-1],dtype = float,sep = ' ')
    vocab.append(w2vdf.iloc[i,1])
#print(g)

In [5]:
datadf = pd.read_csv('../input/data.csv')
datadf = datadf.iloc[:, 1:]
datadf.columns = ['text', 'label']
df = datadf
print(df.head(20))
df.shape

                                                 text  label
0   as of march 13th 2014 the booklet had been dow...      0
1   in order to help increase the booklets downloa...      0
2   simply copy and paste the following text into ...      0
3   click below for a free download of a colorfull...      1
4     click on the download 7 42 mb green banner link      0
5                    booklet updated on feb 14th 2014      0
6                      now with over 18 300 downloads      0
7   then why not simply copy this text links and p...      0
8   thank you in advance download the youtube desc...      0
9   she may or may not be a jew but she s certainl...      1
10  thank you for the info i will deffinetly look ...      0
11  i hope there is room for one more as i am very...      0
12           i just came across this piece of garbage      0
13                        comments say she is from sa      0
14  die antwoord cookie thumper official video you...      0
15    the whites are mor

(9925, 2)

In [6]:
labels = [df.at[i,'label'] for i in range(df.shape[0])]

In [7]:
def cnfmatrix(y_test,results):
    fp = 0.0
    fn = 0.0
    tp = 0.0
    tn = 0.0
    t = 0.0
    n = 0.0
    results.shape
    for i in range(results.shape[0]):
        if y_test[i]==1 and results[i]==1:
            tp+=1
            t+=1
        elif y_test[i]==1 and results[i]==0:
            fn+=1
            t+=1
        elif y_test[i]==0 and results[i]==1:
            fp+=1
            n+=1
        elif y_test[i]==0 and results[i]==0:
            tn+=1
            n+=1
    print(tp/results.shape[0],fp/results.shape[0])
    print(fn/results.shape[0],tn/results.shape[0])
    Precision  = tp/(tp+fp)
    Recall = tp/(tp+fn)
    print("Precision: ",Precision,"Recall: ",Recall)
    f1score = (2*Precision*Recall)/(Precision+Recall)
    print("f1score: ",f1score)
    print("accuracy: ",(tp+tn)/results.shape[0])
    print("hate_acc: ", (tp)/t)
    print("non_hate_acc: ", (tn)/n)

In [8]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
#     stops = set(stopwords.words("english"))
#     text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    #text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", "", text)
    text = re.sub(r"\/", "", text)
    text = re.sub(r"\^", "", text)
    text = re.sub(r"\+", "", text)
    text = re.sub(r"\-", "", text)
    text = re.sub(r"\=", "", text)
    text = re.sub(r"'", " ", text)
    
    return text

In [9]:
df.head()

,text,label
0,as of march 13th 2014 the booklet had been dow...,0
1,in order to help increase the booklets downloa...,0
2,simply copy and paste the following text into ...,0
3,click below for a free download of a colorfull...,1
4,click on the download 7 42 mb green banner link,0


In [10]:
df.iloc[:, 1]

0       0
1       0
2       0
3       1
4       0
5       0
6       0
7       0
8       0
9       1
10      0
11      0
12      0
13      0
14      0
15      0
16      0
17      0
18      0
19      1
20      1
21      0
22      0
23      0
24      0
25      0
26      0
27      0
28      0
29      1
       ..
9895    0
9896    0
9897    0
9898    0
9899    0
9900    0
9901    0
9902    0
9903    0
9904    0
9905    0
9906    0
9907    0
9908    0
9909    0
9910    0
9911    1
9912    0
9913    0
9914    1
9915    0
9916    0
9917    0
9918    0
9919    0
9920    0
9921    0
9922    0
9923    1
9924    0
Name: label, Length: 9925, dtype: int64

In [11]:
data = []
print(data)
for i in range(df.shape[0]):
    b = df.at[i,'text'].split()
    p = []
    for j in b:
        #print(pdict[j])
        p.append(g[j])
    p = p + [g['unk']]*(50-len(p))
    data.append(p)
labels = np.array(labels)
labels.shape
ts = 7000
#print(data[:10])

[]


In [12]:
from sklearn import preprocessing
data = np.array(data,dtype=float)
temp = np.zeros(data.shape)

for i in range(data.shape[0]):
        temp[i] = preprocessing.normalize(data[i])
#print(temp[0][0])
datab = np.array([[vocab.index(j) for j in df.iloc[i,0].split()] for i in range(df.shape[0])])
datab = sequence.pad_sequences(datab, maxlen=50)
xb_train = datab[:ts]
xb_test = datab[ts:]
x_train,y_train = temp[:ts],labels[:ts]
x_test,y_test = temp[ts:],labels[ts:]
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)


(7000, 50, 300)
(7000,)
(2925, 50, 300)
(2925,)


In [13]:
xb_train[-1]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     3,  1262,  3052,   300,     4,  4540,
         472,   800,  2203,     1,   397,     6,  3615,    10,    21,
         991,    12,  4540,   472,    27,    62,  1564,   573, 12281,
           2,    24,  1898,    60, 12282], dtype=int32)

In [39]:
from keras.wrappers.scikit_learn import KerasRegressor,KerasClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
embedding_matrix = np.array([g[i] for i in vocab])
def the_models():
    model = Sequential()
    model.add(Embedding(len(vocab), 300, input_length=50, trainable=True))
    #model.add(layers.Dense(50,activation = 'linear'))
    model.add(CuDNNLSTM(50,return_sequences=True))
    model.add(CuDNNLSTM(50))
    model.add(layers.Dense(20,activation = 'relu'))
    model.add(layers.Dense(1,activation = 'sigmoid'))
    #sgd = optimizers.SGD(lr=0.0001, decay=0.001, momentum=0.0, nesterov=True)
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model


#ann_estimator = KerasClassifier(build_fn = the_models, epochs=10, batch_size=20, verbose=True)
#boosted_ann = AdaBoostClassifier(base_estimator=ann_estimator, n_estimators = 1)

#the_models()

In [51]:
model = the_models()
history = model.fit(xb_train,y_train,epochs =8,batch_size=100)
#boosted_ann.fit(xb_train, y_train)

Epoch 1/8
7000/7000 [==============================] - 2s 321us/step - loss: 0.4042 - acc: 0.8723
Epoch 2/8
7000/7000 [==============================] - 1s 105us/step - loss: 0.2910 - acc: 0.8860
Epoch 3/8
7000/7000 [==============================] - 1s 104us/step - loss: 0.1570 - acc: 0.9404
Epoch 4/8
7000/7000 [==============================] - 1s 102us/step - loss: 0.0806 - acc: 0.9731
Epoch 5/8
7000/7000 [==============================] - 1s 104us/step - loss: 0.0482 - acc: 0.9837
Epoch 6/8
7000/7000 [==============================] - 1s 104us/step - loss: 0.0291 - acc: 0.9910
Epoch 7/8
7000/7000 [==============================] - 1s 103us/step - loss: 0.0189 - acc: 0.9954
Epoch 8/8
7000/7000 [==============================] - 1s 105us/step - loss: 0.0105 - acc: 0.9981


In [53]:
predictions = boosted_ann.predict_proba(xb_test)
#results = results.tolist()
results = [int(i[0]<i[1]) for i in predictions]
print(len(results))

2925/2925 [==============================] - 1s 172us/step
2925


In [3]:
# uncumment below lines to show loss graph
# %matplotlib inline
# import mpld3
# mpld3.enable_notebook()
# plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
# plt.show()

In [56]:
cnfmatrix(y_test,np.array(results))

0.046495726495726496 0.057777777777777775
0.06358974358974359 0.8321367521367521
Precision:  0.4459016393442623 Recall:  0.422360248447205
f1score:  0.43381180223285487
accuracy:  0.8786324786324786
hate_acc:  0.422360248447205
non_hate_acc:  0.9350749135612755
